In [2]:
!pip install plotly streamlit streamlit-keplergl

  Using cached plotly-6.5.2-py3-none-any.whl.metadata (8.5 kB)
  Using cached streamlit-1.54.0-py3-none-any.whl.metadata (9.8 kB)
  Using cached narwhals-2.16.0-py3-none-any.whl.metadata (14 kB)
  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.6-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached protobuf-6.33.5-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using cached tenacity-9.1.4-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using c

In [14]:
!pip install "jupyterlab>=3" "ipywidgets>=7.6"

In [17]:
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
# If that doesn't work, try: 
pio.renderers.default = 'iframe'

In [10]:
import pandas as pd

# Load the data (assuming df is already in memory from your previous check)
# Filter for the specific columns we need
reduced_df = df[['date', 'start_station_name', 'avgTemp', 'value']]

# Save it for the dashboard
reduced_df.to_csv('reduced_data_to_plot.csv', index=False)

# Create the Top 20 dataset
df_groupby_bar = reduced_df.groupby('start_station_name', as_index=False).agg({'value' : 'sum'})
top20 = df_groupby_bar.nlargest(20, 'value')
top20.to_csv('top20.csv', index=False)

print("Summary files 'reduced_data_to_plot.csv' and 'top20.csv' have been created!")

Summary files 'reduced_data_to_plot.csv' and 'top20.csv' have been created!


In [18]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
    x = top20['start_station_name'], 
    y = top20['value'], 
    marker = {'color': top20['value'], 'colorscale': 'Blues'}
))

fig.update_layout(
    title = 'Top 20 Most Popular Citi Bike Stations in NYC (2022)',
    xaxis_title = 'Start Stations',
    yaxis_title = 'Total Trips',
    width = 900, height = 600
)

fig.show()

In [19]:
from plotly.subplots import make_subplots

# Aggregate by date for the line chart
df_daily = reduced_df.groupby('date').agg({'value': 'sum', 'avgTemp': 'mean'}).reset_index()

# Create figure with secondary y-axis
fig_2 = make_subplots(specs=[[{"secondary_y": True}]])

# Add trace for Daily Bike Rides (Primary Axis)
fig_2.add_trace(
    go.Scatter(x=df_daily['date'], y=df_daily['value'], name='Daily Bike Rides', 
               line=dict(color='blue', width=2)),
    secondary_y=False
)

# Add trace for Daily Temperature (Secondary Axis)
fig_2.add_trace(
    go.Scatter(x=df_daily['date'], y=df_daily['avgTemp'], name='Daily Temperature', 
               line=dict(color='red', width=2, dash='dot')),
    secondary_y=True
)

fig_2.update_layout(
    title_text='Daily Citi Bike Rides vs. Average Temperature (NYC 2022)',
    xaxis_title='Date',
    height=600
)

# Label axes
fig_2.update_yaxes(title_text="Number of Rides", secondary_y=False)
fig_2.update_yaxes(title_text="Temperature (°F)", secondary_y=True)

fig_2.show()

In [22]:
import pandas as pd
# Load your local large file (from wherever you moved it)
df_temp = pd.read_csv('reduced_data_to_plot.csv')

# Aggregating to DAILY level (This makes the file tiny, < 1MB)
daily_summarized = df_temp.groupby('date').agg({'value': 'sum', 'avgTemp': 'mean'}).reset_index()
daily_summarized.to_csv('reduced_data_to_plot.csv', index=False)